# Learning about the CELLxGENE Cell Census

The Cell Census is a versioned container for the single-cell data hosted at [CELLxGENE Discover](https://cellxgene.cziscience.com/). The Cell Census utilizes [SOMA](https://github.com/single-cell-data/SOMA/blob/main/abstract_specification.md) powered by [TileDB](https://tiledb.com/products/tiledb-embedded) for storing, accessing, and efficiently filtering data.

This notebook shows you how to learn about the Cell Census contents and how to query it.




# Contents
- Opening the census
- Cell Census organization
- Cell Census summary content tables
- Cell metadata
- Gene metadata
- Understanding Cell Census contents beyond the summary tables

## Opening the census

The `cell_census` python package contains a convenient API to open the latest version of the Cell Census.

In [1]:
import cell_census

census = cell_census.open_soma()

You can learn more about the `cell_census` methods by accessing their corresponding documentation via `help()`. For example `help(cell_census.open_soma)`.

## Cell Census organization

The Cell Census schema[INSERT LINK] defines the structure of the Cell Census. In short, you can think of the Cell Census as a structured collection of items that stores different pieces of information. All of these items and the parent collection are SOMA objects of various types and can all be accessed with the `SOMA` API.

The `cell_census` package contains some convenient wrappers of the SOMA API. An example of this is the function we used to open the Cell Census: `cell_census.open_soma()`

### Main Cell Census components

With the command above you created `census`, which is a `SOMACollection`. It is analogous to a Python dictionary, and it has two items: `census_info` and `census_data`.


#### Cell Census summary info

- `census["census_info"]` A collection of tables providing information of the census as a whole.
    - `census["census_info"]["summary"]`: High-level information of this Cell Census, e.g. build date, total cell count, etc.
    - `census["census_info"]["datasets"]`: A table with all datasets from [CELLxGENE Discover](https://cellxgene.cziscience.com/) used to create the Cell Census.
    - `census["census_info"]["summary_cell_counts"]`: Cell Counts stratified by **relevant** cell metadata
    
    
#### Cell Census data

Data for each organism is stored in independent `SOMAExperiment` objects which are a specialized form of a `SOMACollection`. Each of these store a data matrix (cell by genes), cell metadata, gene metadata, and some other useful components not covered in this notebook.

This is how the data is organized for one organism -- *Homo sapiens*:

- `census_obj["census_data"]["homo_sapiens"].obs`: Cell metadata
- `census_obj["census_data"]["homo_sapiens"].ms["RNA"].X:` Data matrices, currently only raw counts exist `X["raw"]`
- `census_obj["census_data"]["homo_sapiens"].ms["RNA"].var:` Gene Metadata

## Cell Census summary content tables

You can take a quick look at the high-level Cell Census information by looking at `census["census_info"]["summary"]`

In [2]:
census_info = census["census_info"]["summary"].read().concat().to_pandas()
census_info

,soma_joinid,label,value
0,0,cell_census_schema_version,0.1.0
1,1,cell_census_build_date,2023-01-13
2,2,total_cell_count,47129886
3,3,unique_cell_count,25213273
4,4,number_donors_homo_sapiens,9543
5,5,number_donors_mus_musculus,1296


Of special interest are the `label`-`value` combinations for :

- `total_cell_count` is the total number of cells in the Cell Census.
- `unique_cell_count` is the number of unique cells, as some cells may be present twice due to meta-analysis or consortia-like data.
- `number_donors_homo_sapiens` and `number_donors_mus_musculus` are the number of individuals for human and mouse. These are not guaranteed to be unique as one individual ID may be present or identical in different datasets.


###  Cell counts by cell metadata

By looking at `census["summary_cell_counts"]` you can get a general idea of cell counts stratified by **some relevant** cell metadata. Not all cell metadata is included in this table, you can take a look at all cell and gene metadata available in the sections below "Cell metadata" and "Gene metadata".

The line below retrieves this table and casts it into a `pandas.DataFrame`.

In [3]:
census_counts = census["census_info"]["summary_cell_counts"].read().concat().to_pandas()
census_counts

,soma_joinid,organism,category,ontology_term_id,unique_cell_count,total_cell_count,label
0,0,Homo sapiens,all,na,22886364,43207796,na
1,1,Homo sapiens,assay,EFO:0008722,177719,260396,Drop-seq
2,2,Homo sapiens,assay,EFO:0008780,0,51304,inDrop
3,3,Homo sapiens,assay,EFO:0008913,133511,133511,single-cell RNA sequencing
4,4,Homo sapiens,assay,EFO:0008919,44721,161998,Seq-Well
...,...,...,...,...,...,...,...
1198,1198,Mus musculus,tissue_general,UBERON:0002113,164881,188361,kidney
1199,1199,Mus musculus,tissue_general,UBERON:0002365,15577,31154,exocrine gland
1200,1200,Mus musculus,tissue_general,UBERON:0002367,37715,130135,prostate gland
1201,1201,Mus musculus,tissue_general,UBERON:0002368,13322,26644,endocrine gland


For each combination of `organism` and values for each `category` of  cell metadata you can take a look at `total_cell_count` and `unique_cell_count` for the cell counts of that combination. 

The values for each `category` are specified in `ontology_term_id` and `label`, which are the value's IDs and labels, respectively.

#### Example: cell metadata included in the summary counts table

To get all the available cell metadata in the summary counts table you can do the following. Remember this is not all the cell metadata available, as some variables were omitted in the creation of this table.

In [4]:
census_counts[["organism", "category"]].value_counts(sort=False)

organism      category               
Homo sapiens  all                          1
              assay                       20
              cell_type                  557
              disease                     51
              self_reported_ethnicity     16
              sex                          3
              suspension_type              1
              tissue                     196
              tissue_general              59
Mus musculus  all                          1
              assay                        9
              cell_type                  211
              disease                      5
              self_reported_ethnicity      1
              sex                          3
              suspension_type              1
              tissue                      45
              tissue_general              23
dtype: int64

#### Example: cell counts for each sequencing assay in human data 

To get the cell counts for each sequencing assay type in human data, you can perform the following `pandas.DataFrame` operations:

In [5]:
census_human_assays = census_counts.query(
    "organism == 'Homo sapiens' & category == 'assay'"
)
census_human_assays.sort_values("total_cell_count", ascending=False)

,soma_joinid,organism,category,ontology_term_id,unique_cell_count,total_cell_count,label
11,11,Homo sapiens,assay,EFO:0009922,5714414,17962192,10x 3' v3
8,8,Homo sapiens,assay,EFO:0009899,5686921,10599706,10x 3' v2
14,14,Homo sapiens,assay,EFO:0010550,4062980,5064268,sci-RNA-seq
15,15,Homo sapiens,assay,EFO:0011025,3289276,4684103,10x 5' v1
9,9,Homo sapiens,assay,EFO:0009900,1611873,1740273,10x 5' v2
17,17,Homo sapiens,assay,EFO:0030003,651223,657933,10x 3' transcription profiling
16,16,Homo sapiens,assay,EFO:0030002,598266,642559,microwell-seq
18,18,Homo sapiens,assay,EFO:0030004,600929,600929,10x 5' transcription profiling
1,1,Homo sapiens,assay,EFO:0008722,177719,260396,Drop-seq
19,19,Homo sapiens,assay,EFO:0700003,136067,177276,BD Rhapsody Whole Transcriptome Analysis


#### Example: number of microglial cells in the Cell Census

If you have a specific term from any of the categories shown above you can directly find out the number of cells for that term.

In [6]:
census_counts.query("label == 'microglial cell'")

,soma_joinid,organism,category,ontology_term_id,unique_cell_count,total_cell_count,label
65,65,Homo sapiens,cell_type,CL:0000129,68691,326622,microglial cell
936,936,Mus musculus,cell_type,CL:0000129,34990,62617,microglial cell


## Cell metadata

You can obtain all cell metadata variables by directly querying the columns of the corresponding `SOMADataFrame`.

All of these variables can be used for querying the Cell Census in case you want to work with specific cells.

In [7]:
list(census["census_data"]["homo_sapiens"].obs.keys())

['soma_joinid',
 'dataset_id',
 'assay',
 'assay_ontology_term_id',
 'cell_type',
 'cell_type_ontology_term_id',
 'development_stage',
 'development_stage_ontology_term_id',
 'disease',
 'disease_ontology_term_id',
 'donor_id',
 'is_primary_data',
 'self_reported_ethnicity',
 'self_reported_ethnicity_ontology_term_id',
 'sex',
 'sex_ontology_term_id',
 'suspension_type',
 'tissue',
 'tissue_ontology_term_id',
 'tissue_general',
 'tissue_general_ontology_term_id']

All of these variables are defined in the [CELLxGENE dataset schema](https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/3.0.0/schema.md#obs-cell-metadata) except for the following:

- `soma_joinid`: a SOMA-defined value use for join operations.
- `dataset_id`: the dataset id as encoded in `census["census-info"]["datasets"]`.
- `tissue_general` and `tissue_general_ontology_term_id`: the high-level tissue mapping.

## Gene metadata

Similarly, we can obtain all gene metadata variables by directly querying the columns of the corresponding `SOMADataFrame`.

These are the variables you can use for querying the Cell Census in case there are specific genes you are interested in.

In [8]:
list(census["census_data"]["homo_sapiens"].ms["RNA"].var.keys())

['soma_joinid', 'feature_id', 'feature_name', 'feature_length']

All of these variables are defined in the [CELLxGENE dataset schema](https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/3.0.0/schema.md#var-and-rawvar-gene-metadata) except for the following:

- `soma_joinid`: a SOMA-defined value use for join operations.
- `feature_length`: the length in base pairs of the gene.

## Understanding Cell Census contents beyond the summary tables

While using the pre-computed tables in `census["census_info"]` is an easy and quick way to understand the contents of the Cell Census, it falls short if you want to learn more about certain slices of the Cell Census.

For example, you may want to learn more about:

- What are the cell types available for human liver? 
- What are the total number of cells in all lung datasets stratified by sequencing technology?
- What is the sex distribution of all cells from brain in mouse?
- What are the diseases available for T cells? 

All of these questions can be answered by directly querying the cell metadata as shown in the examples below.

### Example: all cell types available in human

To exemplify the process of accessing and slicing cell metadata for summary stats, let's start with a trivial example and take a look at all human cell types available in the Cell Census:

In [9]:
human_cell_types = (
    census["census_data"]["homo_sapiens"]
    .obs.read(column_names=["cell_type", "is_primary_data"])
    .concat()
    .to_pandas()
)
human_cell_types

,cell_type,is_primary_data
0,tracheal goblet cell,True
1,memory B cell,True
2,gamma-delta T cell,True
3,"CD8-positive, alpha-beta memory T cell",True
4,tracheal goblet cell,True
...,...,...
43207791,mesodermal cell,True
43207792,colon epithelial cell,True
43207793,enteroendocrine cell,True
43207794,colon epithelial cell,True


The number of rows is the total number of cells for humans. Now, if you wish to get the cell counts per cell type we can perform some `pandas` operations on this object.

In addition, we will only focus on cells that are marked with `is_primary_data=True` as this ensures we de-duplicate cells that appear more than once in CELLxGENE Discover.

In [10]:
human_cell_types = (
    census["census_data"]["homo_sapiens"]
    .obs.read(column_names=["cell_type"], value_filter="is_primary_data == True")
    .concat()
    .to_pandas()
)

human_cell_types = human_cell_types[["cell_type"]]
human_cell_types.shape

(22886364, 1)

This is the number of unique cells. Now let's look at the counts per cell type:

In [11]:
human_cell_type_counts = human_cell_types.value_counts()
human_cell_type_counts

cell_type                            
glutamatergic neuron                     1546910
CD8-positive, alpha-beta T cell           955814
CD4-positive, alpha-beta T cell           946341
classical monocyte                        919044
B cell                                    789889
                                          ...   
mature conventional dendritic cell            17
serous cell of epithelium of bronchus         15
sperm                                         11
type N enteroendocrine cell                   10
mature T cell                                  1
Length: 539, dtype: int64

This shows you that the most abundant cell types are "glutamatergic neuron", "CD8-positive, alpha-beta T cell", and "CD4-positive, alpha-beta T cell". 

Now let's take a look at the number of unique cell types:

In [12]:
human_cell_type_counts.shape

(539,)

That is the total number of different cell types for human.

All the information in this example can be quickly obtained from the summary table at `census["census-info"]["summary_cell_counts"]`.

The examples below are more complex and can only be achieved by accessing the cell metadata.

### Example: cell types available in human liver

Similar to the example above, we can learn what cell types are available for a specific tissue, e.g. liver.

To achieve this goal we just need to limit our cell metadata to that tissue. We will use the information in the cell metadata variable `tissue_general`. This variable contains the high-level tissue label for all cells in the Cell Census:

In [13]:
human_liver_cell_types = (
    census["census_data"]["homo_sapiens"]
    .obs.read(
        column_names=["cell_type"],
        value_filter="is_primary_data == True and tissue_general == 'liver'",
    )
    .concat()
    .to_pandas()
)

human_liver_cell_types["cell_type"].value_counts()

erythrocyte                   63099
hepatoblast                   58447
macrophage                    57741
neoplastic cell               52431
erythroblast                  42328
                              ...  
chondrocyte                       1
interstitial cell of Cajal        1
endocrine cell                    1
cell of skeletal muscle           1
germinal center B cell            1
Name: cell_type, Length: 121, dtype: int64

These are the cell types and their cell counts in the human liver.

### Example: diseased T cells in human tissues

In this example we are going to get the counts for all diseased cells annotated as T cells. For the sake of the example we will focus on "CD8-positive, alpha-beta T cell" and "CD4-positive, alpha-beta T cell":

In [14]:
t_cells_list = ["CD8-positive, alpha-beta T cell", "CD4-positive, alpha-beta T cell"]

t_cells_diseased = (
    census["census_data"]["homo_sapiens"]
    .obs.read(
        column_names=["disease", "tissue_general"],
        value_filter=f"is_primary_data == True and cell_type in {t_cells_list} and disease != 'normal'",
    )
    .concat()
    .to_pandas()
)

t_cells_diseased = t_cells_diseased[["disease", "tissue_general"]].value_counts(
    sort=False
)
t_cells_diseased

disease                       tissue_general    
B-cell non-Hodgkin lymphoma   blood                  62499
COVID-19                      blood                 814620
                              lung                    5014
                              nose                      13
                              respiratory system         4
                              saliva                    41
Crohn disease                 small intestine          960
Down syndrome                 bone marrow              181
follicular lymphoma           lymph node              1089
influenza                     blood                   8871
lung adenocarcinoma           lung                   30499
respiratory system disorder   blood                  34301
squamous cell lung carcinoma  lung                    9001
systemic lupus erythematosus  blood                 355471
dtype: int64

These are the cell counts annotated with the indicated disease across human tissues for "CD8-positive, alpha-beta T cell" or "CD4-positive, alpha-beta T cell".